# SageMaker Coding Agent - Complete Single-File Version

A **secure** AI coding assistant powered by AWS Bedrock Claude.

## Features
- **14 Tools**: File ops, search, bash, Python execution, documents, vision, todos
- **Security**: Workspace boundary, secret detection, command filtering, audit logging
- **Session Management**: Save/load conversations
- **Context Warnings**: Alerts at 80%, 90%, 95% context usage
- **Approval System**: User approval for write/execute operations

## Quick Start
1. Run all cells in order (Shift+Enter)
2. Use the chat widget at the bottom
3. Click Approve/Deny for write operations

In [ ]:
# ============================================================
# CELL 1: INSTALL DEPENDENCIES
# ============================================================
!pip install -q boto3 ipywidgets Pillow python-docx pandas openpyxl

In [ ]:
# ============================================================
# CELL 2: CONFIGURATION
# ============================================================

import os
import json
import re
import hashlib
import subprocess
import tempfile
import sys
import base64
import time
from dataclasses import dataclass, asdict, field
from typing import List, Dict, Tuple, Optional, Any, Set, Callable
from datetime import datetime
from pathlib import Path
from collections import deque
import glob as glob_module

@dataclass
class Config:
    """Agent configuration."""
    # AWS Settings
    region: str = "ap-southeast-2"  # Sydney
    model_id: str = "anthropic.claude-3-5-sonnet-20241022-v2:0"
    
    # Workspace
    workspace: str = "."
    sessions_dir: str = "./sessions"
    audit_dir: str = "./audit_logs"
    
    # Limits
    max_turns: int = 30
    max_tokens: int = 4096
    max_history: int = 20
    max_output_chars: int = 5000
    max_file_size: int = 10 * 1024 * 1024  # 10MB
    
    # Context limits (Claude 3.5 = 200K tokens)
    context_max_tokens: int = 200000
    
    # Testing
    mock_mode: bool = False  # Set True to test without Bedrock API

# Initialize config
CONFIG = Config()

# Create directories
os.makedirs(CONFIG.sessions_dir, exist_ok=True)
os.makedirs(CONFIG.audit_dir, exist_ok=True)

print(f"Config loaded:")
print(f"  Region: {CONFIG.region}")
print(f"  Model: {CONFIG.model_id[:40]}...")
print(f"  Mock mode: {CONFIG.mock_mode}")
print(f"  Workspace: {os.path.abspath(CONFIG.workspace)}")

In [ ]:
# ============================================================
# CELL 3: SECURITY MODULE
# ============================================================

class SecurityManager:
    """Security controls: workspace boundary, secret detection, command filtering."""
    
    SECRET_PATTERNS = [
        (r"(?i)(api[_-]?key|apikey)\s*[=:]\s*[\"']?[\w-]{20,}", "API Key"),
        (r"(?i)(secret|password|passwd|pwd)\s*[=:]\s*[\"']?[^\s\"']{8,}", "Password/Secret"),
        (r"(?i)(aws[_-]?access[_-]?key[_-]?id)\s*[=:]\s*[\"']?[A-Z0-9]{20}", "AWS Access Key"),
        (r"(?i)(aws[_-]?secret[_-]?access[_-]?key)\s*[=:]\s*[\"']?[A-Za-z0-9/+=]{40}", "AWS Secret Key"),
        (r"(?i)(bearer\s+)[A-Za-z0-9\-_]+\.[A-Za-z0-9\-_]+\.[A-Za-z0-9\-_]+", "JWT Token"),
        (r"-----BEGIN (RSA |DSA |EC |OPENSSH )?PRIVATE KEY-----", "Private Key"),
        (r"(?i)(mongodb|postgres|mysql|redis)://[^\s]+:[^\s]+@", "Database URL"),
        (r"(?i)(gh[ps]_[A-Za-z0-9_]{36,})", "GitHub Token"),
        (r"(?i)(xox[baprs]-[A-Za-z0-9-]+)", "Slack Token"),
    ]
    
    SENSITIVE_FILES = {
        ".env", ".env.local", ".env.production", ".env.development",
        "credentials.json", "secrets.json", "config.secret.json",
        "id_rsa", "id_ed25519", "id_dsa", "id_ecdsa",
        ".netrc", ".npmrc", ".pypirc",
    }
    
    DANGEROUS_PATTERNS = [
        (r"\brm\s+-rf\s+/", "Recursive delete from root"),
        (r"\brm\s+-rf\s+\*", "Recursive delete wildcard"),
        (r"\bdd\s+if=", "Direct disk access"),
        (r"\bmkfs", "Filesystem creation"),
        (r"\bcurl\s+.*\|\s*(ba)?sh", "Pipe to shell"),
        (r"\bwget\s+.*\|\s*(ba)?sh", "Pipe to shell"),
        (r"\bchmod\s+777", "Overly permissive chmod"),
        (r"\bsudo\s+", "Sudo command"),
        (r"\b>\s*/dev/sd", "Direct device write"),
        (r"\bnc\s+-l", "Network listener"),
        (r"\beval\s+\$", "Eval with variable"),
        (r"\bbase64\s+-d.*\|\s*(ba)?sh", "Encoded payload execution"),
    ]
    
    NETWORK_COMMANDS = ["curl", "wget", "nc", "netcat", "ssh", "scp", "rsync", "ftp", "telnet"]
    
    def __init__(self, workspace: str, allow_network: bool = False):
        self.workspace = Path(workspace).resolve()
        self.allow_network = allow_network
    
    def validate_path(self, path: str) -> Tuple[bool, str]:
        """Check if path is within workspace."""
        try:
            if not os.path.isabs(path):
                resolved = (self.workspace / path).resolve()
            else:
                resolved = Path(path).resolve()
            
            try:
                resolved.relative_to(self.workspace)
            except ValueError:
                return False, f"Path outside workspace: {path}"
            
            if resolved.name in self.SENSITIVE_FILES:
                return False, f"Access to sensitive file blocked: {resolved.name}"
            
            for part in resolved.parts:
                if part.startswith(".env"):
                    return False, f"Access to .env file blocked"
            
            return True, "OK"
        except Exception as e:
            return False, f"Invalid path: {e}"
    
    def validate_command(self, command: str) -> Tuple[bool, str]:
        """Check if bash command is safe."""
        for pattern, reason in self.DANGEROUS_PATTERNS:
            if re.search(pattern, command, re.IGNORECASE):
                return False, f"Blocked: {reason}"
        
        if not self.allow_network:
            for cmd in self.NETWORK_COMMANDS:
                if re.search(rf"\b{cmd}\b", command):
                    return False, f"Network command blocked: {cmd}"
        
        return True, "OK"
    
    def scan_secrets(self, content: str) -> List[Dict]:
        """Scan for potential secrets."""
        findings = []
        for pattern, secret_type in self.SECRET_PATTERNS:
            try:
                matches = re.findall(pattern, content)
                if matches:
                    findings.append({"type": secret_type, "count": len(matches)})
            except re.error:
                continue
        return findings
    
    def truncate_output(self, output: str, max_size: int = None) -> str:
        """Truncate output to prevent token explosion."""
        max_size = max_size or CONFIG.max_output_chars
        if len(output) <= max_size:
            return output
        return output[:max_size] + f"\n[Truncated - {len(output):,} chars total]"

# Initialize security
SECURITY = SecurityManager(CONFIG.workspace)
print(f"Security initialized. Workspace: {SECURITY.workspace}")

In [ ]:
# ============================================================
# CELL 4: AUDIT LOGGING
# ============================================================

@dataclass
class AuditEntry:
    """Single audit log entry."""
    timestamp: str
    session_id: str
    action: str
    tool_name: Optional[str]
    parameters: Dict[str, Any]
    result_summary: str
    user_approved: bool
    hash: str = ""
    
    def __post_init__(self):
        if not self.hash:
            content = f"{self.timestamp}|{self.session_id}|{self.action}|{self.tool_name}|{self.result_summary}"
            self.hash = hashlib.sha256(content.encode()).hexdigest()[:32]

class AuditLogger:
    """Immutable audit trail with integrity verification."""
    
    SENSITIVE_KEYS = {"password", "secret", "key", "token", "credential", "api_key", "auth", "bearer", "private"}
    
    def __init__(self, audit_dir: str):
        self.audit_dir = audit_dir
        os.makedirs(audit_dir, exist_ok=True)
    
    def _get_log_path(self, session_id: str) -> str:
        date = datetime.now().strftime("%Y-%m-%d")
        return os.path.join(self.audit_dir, f"{date}_{session_id}.jsonl")
    
    def log(self, session_id: str, action: str, tool_name: str = None, 
            parameters: Dict = None, result_summary: str = "", user_approved: bool = True):
        """Log an action to audit trail."""
        entry = AuditEntry(
            timestamp=datetime.now().isoformat(),
            session_id=session_id,
            action=action,
            tool_name=tool_name,
            parameters=self._sanitize_params(parameters or {}),
            result_summary=result_summary[:500] if result_summary else "",
            user_approved=user_approved,
        )
        log_path = self._get_log_path(session_id)
        with open(log_path, "a", encoding="utf-8") as f:
            f.write(json.dumps(asdict(entry)) + "\n")
    
    def _sanitize_params(self, params: Dict) -> Dict:
        """Remove sensitive data from parameters."""
        sanitized = {}
        for k, v in params.items():
            if any(s in k.lower() for s in self.SENSITIVE_KEYS):
                sanitized[k] = "[REDACTED]"
            elif isinstance(v, str) and len(v) > 1000:
                sanitized[k] = f"[{len(v)} chars]"
            else:
                sanitized[k] = v
        return sanitized
    
    def get_session_log(self, session_id: str) -> List[Dict]:
        """Get all entries for a session."""
        entries = []
        for filename in os.listdir(self.audit_dir):
            if session_id in filename and filename.endswith(".jsonl"):
                path = os.path.join(self.audit_dir, filename)
                with open(path, "r", encoding="utf-8") as f:
                    for line in f:
                        if line.strip():
                            entries.append(json.loads(line))
        return sorted(entries, key=lambda x: x.get("timestamp", ""))

# Initialize audit
AUDIT = AuditLogger(CONFIG.audit_dir)
print(f"Audit logger ready. Directory: {CONFIG.audit_dir}")

In [ ]:
# ============================================================
# CELL 5: BEDROCK CLIENT
# ============================================================

import boto3

@dataclass
class ToolCall:
    id: str
    name: str
    input: dict

@dataclass
class Response:
    text: str
    tool_calls: List[ToolCall]
    stop_reason: str
    usage: dict

class BedrockClient:
    """AWS Bedrock Claude client with mock mode for testing."""
    
    def __init__(self, model_id: str, region: str, mock_mode: bool = False):
        self.model_id = model_id
        self.region = region
        self.mock_mode = mock_mode
        if not mock_mode:
            self.client = boto3.client("bedrock-runtime", region_name=region)
        else:
            self.client = None
            print("[MOCK MODE] No API calls will be made")
    
    def _mock_response(self, messages, tools) -> Response:
        """Generate mock response for testing."""
        last = messages[-1]["content"] if messages else ""
        if isinstance(last, str):
            low = last.lower()
            if "list" in low and ("file" in low or "dir" in low):
                return Response("I'll list the files.", [ToolCall("m1", "list_dir", {"path": "."})], "tool_use", {})
            if "read" in low:
                return Response("I'll read that file.", [ToolCall("m2", "read_file", {"file_path": "README.md"})], "tool_use", {})
            if "create" in low and "excel" in low:
                return Response("I'll create an Excel file.", [ToolCall("m3", "create_excel", {"filepath": "test.xlsx", "data": [{"name": "Alice", "age": 30}]})], "tool_use", {})
            if "todo" in low:
                return Response("I'll track these tasks.", [ToolCall("m4", "todo_write", {"todos": [{"content": "Sample task", "status": "pending", "activeForm": "Working on sample task"}]})], "tool_use", {})
        return Response(f"[MOCK] Received: {str(last)[:100]}...", [], "end_turn", {"input_tokens": 100, "output_tokens": 50})
    
    def chat(self, messages: List[Dict], system: str, tools: List[Dict] = None, max_tokens: int = 4096) -> Response:
        """Send chat request to Bedrock."""
        if self.mock_mode:
            return self._mock_response(messages, tools)
        
        body = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "system": system,
            "messages": messages,
            "temperature": 0.0,
        }
        if tools:
            body["tools"] = tools
        
        response = self.client.invoke_model(
            modelId=self.model_id,
            body=json.dumps(body),
            contentType="application/json"
        )
        result = json.loads(response["body"].read())
        return self._parse(result)
    
    def _parse(self, result: dict) -> Response:
        """Parse Bedrock response."""
        text = ""
        tool_calls = []
        for block in result.get("content", []):
            if block.get("type") == "text":
                text += block.get("text", "")
            elif block.get("type") == "tool_use":
                tool_calls.append(ToolCall(
                    id=block.get("id", ""),
                    name=block.get("name", ""),
                    input=block.get("input", {})
                ))
        return Response(text, tool_calls, result.get("stop_reason", ""), result.get("usage", {}))

# Initialize client
CLIENT = BedrockClient(CONFIG.model_id, CONFIG.region, CONFIG.mock_mode)
print(f"Bedrock client ready. Mock mode: {CONFIG.mock_mode}")

In [ ]:
# ============================================================
# CELL 6: SESSION MANAGEMENT
# ============================================================

@dataclass
class Session:
    """Conversation session."""
    id: str
    created_at: str
    updated_at: str
    title: str
    messages: List[Dict] = field(default_factory=list)
    metadata: Dict = field(default_factory=dict)

class SessionManager:
    """Persistent session storage."""
    
    def __init__(self, sessions_dir: str):
        self.sessions_dir = sessions_dir
        os.makedirs(sessions_dir, exist_ok=True)
    
    def create(self, title: str = "New Session") -> Session:
        """Create new session."""
        session_id = datetime.now().strftime("%Y%m%d_%H%M%S")
        now = datetime.now().isoformat()
        session = Session(id=session_id, created_at=now, updated_at=now, title=title, messages=[], metadata={})
        self.save(session)
        return session
    
    def save(self, session: Session):
        """Save session to disk."""
        session.updated_at = datetime.now().isoformat()
        path = os.path.join(self.sessions_dir, f"{session.id}.json")
        with open(path, "w", encoding="utf-8") as f:
            json.dump(asdict(session), f, indent=2)
    
    def load(self, session_id: str) -> Optional[Session]:
        """Load session from disk."""
        path = os.path.join(self.sessions_dir, f"{session_id}.json")
        if not os.path.exists(path):
            return None
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        return Session(**data)
    
    def list_sessions(self) -> List[Dict]:
        """List all sessions."""
        sessions = []
        for filename in os.listdir(self.sessions_dir):
            if filename.endswith(".json"):
                try:
                    with open(os.path.join(self.sessions_dir, filename), "r", encoding="utf-8") as f:
                        data = json.load(f)
                    sessions.append({"id": data["id"], "title": data["title"], "updated_at": data["updated_at"]})
                except:
                    continue
        return sorted(sessions, key=lambda x: x.get("updated_at", ""), reverse=True)
    
    def delete(self, session_id: str) -> bool:
        """Delete session."""
        path = os.path.join(self.sessions_dir, f"{session_id}.json")
        if os.path.exists(path):
            os.remove(path)
            return True
        return False

# Initialize session manager
SESSIONS = SessionManager(CONFIG.sessions_dir)
print(f"Session manager ready. Directory: {CONFIG.sessions_dir}")

In [ ]:
# ============================================================
# CELL 7: CONTEXT MANAGER
# ============================================================

class ContextManager:
    """Monitors context usage and provides warnings."""
    
    def __init__(self, max_tokens: int = 200000):
        self.max_tokens = max_tokens
        self.last_warning_level = 0
    
    def estimate_tokens(self, messages: List[Dict]) -> int:
        """Estimate tokens (4 chars = 1 token)."""
        total_chars = 0
        for m in messages:
            content = m.get("content", "")
            if isinstance(content, str):
                total_chars += len(content)
            elif isinstance(content, list):
                for block in content:
                    if isinstance(block, dict):
                        total_chars += len(str(block.get("text", "")))
                        total_chars += len(str(block.get("content", "")))
                    else:
                        total_chars += len(str(block))
            else:
                total_chars += len(str(content))
        return total_chars // 4
    
    def get_usage(self, messages: List[Dict]) -> Dict:
        """Get context usage stats."""
        tokens = self.estimate_tokens(messages)
        percent = tokens / self.max_tokens
        return {
            "tokens": tokens,
            "max_tokens": self.max_tokens,
            "percent": percent,
            "level": "critical" if percent >= 0.95 else "high" if percent >= 0.90 else "medium" if percent >= 0.80 else "normal"
        }
    
    def check_and_warn(self, messages: List[Dict]) -> Optional[str]:
        """Check context and return warning if needed."""
        usage = self.get_usage(messages)
        percent = usage["percent"]
        tokens = usage["tokens"]
        
        if percent >= 0.95 and self.last_warning_level < 95:
            self.last_warning_level = 95
            return f"[!] Context at 95% ({tokens:,}/{self.max_tokens:,} tokens). Compaction imminent!"
        elif percent >= 0.90 and self.last_warning_level < 90:
            self.last_warning_level = 90
            return f"[!] Context at 90% ({tokens:,}/{self.max_tokens:,} tokens). Approaching limit."
        elif percent >= 0.80 and self.last_warning_level < 80:
            self.last_warning_level = 80
            return f"[i] Context at 80% ({tokens:,}/{self.max_tokens:,} tokens). Consider starting fresh."
        return None
    
    def reset(self):
        """Reset warning level."""
        self.last_warning_level = 0

# Initialize context manager
CONTEXT = ContextManager(CONFIG.context_max_tokens)
print(f"Context manager ready. Max tokens: {CONFIG.context_max_tokens:,}")

In [ ]:
# ============================================================
# CELL 8: ALL TOOLS (14 TOOLS)
# ============================================================

# Global state
_TODOS = []
_FILES_READ = set()  # Track files that have been read

# ============== FILE OPERATIONS ==============

def tool_read_file(args: Dict) -> str:
    """Read a file with line numbers."""
    path = args["file_path"]
    offset = args.get("offset", 0)
    limit = args.get("limit", 500)
    
    ok, msg = SECURITY.validate_path(path)
    if not ok:
        return f"Error: {msg}"
    
    if not os.path.isabs(path):
        path = os.path.join(CONFIG.workspace, path)
    
    if not os.path.exists(path):
        return f"Error: File not found: {path}"
    
    try:
        with open(path, 'r', encoding='utf-8', errors='replace') as f:
            lines = f.readlines()
        
        _FILES_READ.add(os.path.abspath(path))
        selected = lines[offset:offset + limit]
        result = []
        for i, line in enumerate(selected, start=offset + 1):
            if len(line) > 2000:
                line = line[:2000] + "...\n"
            result.append(f"{i:4}| {line.rstrip()}")
        
        output = "\n".join(result)
        return SECURITY.truncate_output(output)
    except Exception as e:
        return f"Error reading file: {e}"

def tool_write_file(args: Dict) -> str:
    """Write content to file."""
    path = args["file_path"]
    content = args["content"]
    
    ok, msg = SECURITY.validate_path(path)
    if not ok:
        return f"Error: {msg}"
    
    if not os.path.isabs(path):
        path = os.path.join(CONFIG.workspace, path)
    
    # Check if file exists and was read
    if os.path.exists(path) and os.path.abspath(path) not in _FILES_READ:
        return "Error: Must read file before writing. Use read_file first."
    
    # Scan for secrets
    secrets = SECURITY.scan_secrets(content)
    if secrets:
        types = ", ".join(s["type"] for s in secrets)
        return f"Warning: Content contains potential secrets ({types}). Review before saving."
    
    try:
        dir_path = os.path.dirname(path)
        if dir_path:
            os.makedirs(dir_path, exist_ok=True)
        with open(path, 'w', encoding='utf-8') as f:
            f.write(content)
        _FILES_READ.add(os.path.abspath(path))
        return f"Written {len(content):,} chars to {path}"
    except Exception as e:
        return f"Error writing file: {e}"

def tool_edit_file(args: Dict) -> str:
    """Edit file by replacing exact string."""
    path = args["file_path"]
    old_string = args["old_string"]
    new_string = args["new_string"]
    replace_all = args.get("replace_all", False)
    
    ok, msg = SECURITY.validate_path(path)
    if not ok:
        return f"Error: {msg}"
    
    if not os.path.isabs(path):
        path = os.path.join(CONFIG.workspace, path)
    
    if os.path.abspath(path) not in _FILES_READ:
        return "Error: Must read file before editing. Use read_file first."
    
    try:
        with open(path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        count = content.count(old_string)
        if count == 0:
            return "Error: old_string not found in file. Must be EXACT match."
        if count > 1 and not replace_all:
            return f"Error: old_string appears {count} times. Use replace_all=true or provide more context."
        
        if replace_all:
            new_content = content.replace(old_string, new_string)
        else:
            new_content = content.replace(old_string, new_string, 1)
        
        with open(path, 'w', encoding='utf-8') as f:
            f.write(new_content)
        return f"Edited {path} ({count} replacement{'s' if replace_all and count > 1 else ''})"
    except Exception as e:
        return f"Error editing file: {e}"

def tool_glob(args: Dict) -> str:
    """Find files by glob pattern."""
    pattern = args["pattern"]
    path = args.get("path", CONFIG.workspace)
    
    if not os.path.isabs(path):
        path = os.path.join(CONFIG.workspace, path)
    
    full_pattern = os.path.join(path, pattern)
    matches = glob_module.glob(full_pattern, recursive=True)[:100]
    matches = sorted(matches, key=lambda x: os.path.getmtime(x) if os.path.exists(x) else 0, reverse=True)
    return "\n".join(matches) if matches else "No files found"

def tool_grep(args: Dict) -> str:
    """Search file contents with regex."""
    pattern = args["pattern"]
    path = args.get("path", CONFIG.workspace)
    glob_pattern = args.get("glob", "**/*")
    case_insensitive = args.get("case_insensitive", False)
    
    if not os.path.isabs(path):
        path = os.path.join(CONFIG.workspace, path)
    
    flags = re.IGNORECASE if case_insensitive else 0
    try:
        regex = re.compile(pattern, flags)
    except re.error as e:
        return f"Error: Invalid regex: {e}"
    
    results = []
    files = glob_module.glob(os.path.join(path, glob_pattern), recursive=True)
    
    for filepath in files:
        if not os.path.isfile(filepath) or len(results) >= 50:
            continue
        try:
            with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
                for i, line in enumerate(f, 1):
                    if regex.search(line):
                        results.append(f"{filepath}:{i}: {line.strip()[:100]}")
                        if len(results) >= 50:
                            break
        except:
            continue
    
    return "\n".join(results) if results else "No matches found"

def tool_list_dir(args: Dict) -> str:
    """List directory contents."""
    path = args.get("path", CONFIG.workspace)
    
    if not os.path.isabs(path):
        path = os.path.join(CONFIG.workspace, path)
    
    ok, msg = SECURITY.validate_path(path)
    if not ok:
        return f"Error: {msg}"
    
    if not os.path.isdir(path):
        return f"Error: Not a directory: {path}"
    
    try:
        entries = []
        for entry in sorted(os.listdir(path))[:100]:
            full = os.path.join(path, entry)
            if os.path.isdir(full):
                entries.append(f"[DIR]  {entry}/")
            else:
                size = os.path.getsize(full)
                entries.append(f"[FILE] {entry} ({size:,} bytes)")
        return "\n".join(entries) if entries else "(empty directory)"
    except Exception as e:
        return f"Error: {e}"

# ============== BASH ==============

def tool_bash(args: Dict) -> str:
    """Execute shell command."""
    command = args["command"]
    timeout = min(args.get("timeout", 120), 600)
    
    ok, msg = SECURITY.validate_command(command)
    if not ok:
        return f"Blocked: {msg}"
    
    try:
        result = subprocess.run(
            command,
            shell=True,
            capture_output=True,
            text=True,
            timeout=timeout,
            cwd=CONFIG.workspace,
            env={**os.environ, "TERM": "dumb"}
        )
        output = result.stdout
        if result.stderr:
            output += f"\n[stderr]\n{result.stderr}"
        if result.returncode != 0:
            output += f"\n[exit code: {result.returncode}]"
        return SECURITY.truncate_output(output) if output else "(no output)"
    except subprocess.TimeoutExpired:
        return f"Error: Command timed out after {timeout} seconds"
    except Exception as e:
        return f"Error: {e}"

# ============== PYTHON EXECUTION ==============

def tool_python_exec(args: Dict) -> str:
    """Execute Python code in subprocess."""
    code = args["code"]
    timeout = min(args.get("timeout", 60), 300)
    
    fd, temp_path = tempfile.mkstemp(suffix=".py", prefix="agent_exec_")
    try:
        with os.fdopen(fd, "w", encoding="utf-8") as f:
            f.write(code)
        
        result = subprocess.run(
            [sys.executable, temp_path],
            capture_output=True,
            text=True,
            timeout=timeout,
            cwd=CONFIG.workspace,
            env={**os.environ, "PYTHONIOENCODING": "utf-8"}
        )
        output = result.stdout
        if result.stderr:
            output += f"\n[stderr]\n{result.stderr}"
        if result.returncode != 0:
            output += f"\n[exit code: {result.returncode}]"
        return SECURITY.truncate_output(output) if output else "(no output)"
    except subprocess.TimeoutExpired:
        return f"Error: Code timed out after {timeout} seconds"
    except Exception as e:
        return f"Error: {e}"
    finally:
        try:
            os.unlink(temp_path)
        except:
            pass

# ============== DOCUMENT CREATION ==============

def tool_create_word(args: Dict) -> str:
    """Create Word document."""
    filepath = args["filepath"]
    content = args["content"]
    title = args.get("title", "")
    
    if not os.path.isabs(filepath):
        filepath = os.path.join(CONFIG.workspace, filepath)
    if not filepath.endswith(".docx"):
        filepath += ".docx"
    
    try:
        from docx import Document
        doc = Document()
        if title:
            doc.add_heading(title, 0)
        for para in content.split("\n\n"):
            if para.strip():
                doc.add_paragraph(para.strip())
        dir_path = os.path.dirname(filepath)
        if dir_path:
            os.makedirs(dir_path, exist_ok=True)
        doc.save(filepath)
        return f"Created Word document: {filepath}"
    except ImportError:
        return "Error: python-docx not installed. Run: pip install python-docx"
    except Exception as e:
        return f"Error: {e}"

def tool_create_excel(args: Dict) -> str:
    """Create Excel spreadsheet."""
    filepath = args["filepath"]
    data = args["data"]
    sheet_name = args.get("sheet_name", "Sheet1")
    
    if not os.path.isabs(filepath):
        filepath = os.path.join(CONFIG.workspace, filepath)
    if not filepath.endswith(".xlsx"):
        filepath += ".xlsx"
    
    try:
        import pandas as pd
        df = pd.DataFrame(data)
        dir_path = os.path.dirname(filepath)
        if dir_path:
            os.makedirs(dir_path, exist_ok=True)
        df.to_excel(filepath, sheet_name=sheet_name, index=False)
        return f"Created Excel file: {filepath} ({len(df)} rows)"
    except ImportError:
        return "Error: pandas/openpyxl not installed. Run: pip install pandas openpyxl"
    except Exception as e:
        return f"Error: {e}"

def tool_create_markdown(args: Dict) -> str:
    """Create Markdown file."""
    filepath = args["filepath"]
    content = args["content"]
    
    if not os.path.isabs(filepath):
        filepath = os.path.join(CONFIG.workspace, filepath)
    if not filepath.endswith(".md"):
        filepath += ".md"
    
    try:
        dir_path = os.path.dirname(filepath)
        if dir_path:
            os.makedirs(dir_path, exist_ok=True)
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(content)
        return f"Created Markdown file: {filepath}"
    except Exception as e:
        return f"Error: {e}"

# ============== VISION ==============

def tool_view_image(args: Dict) -> str:
    """Load and describe image for AI analysis."""
    path = args["file_path"]
    
    if not os.path.isabs(path):
        path = os.path.join(CONFIG.workspace, path)
    
    if not os.path.exists(path):
        return f"Error: Image not found: {path}"
    
    ext = os.path.splitext(path)[1].lower()
    media_types = {".png": "image/png", ".jpg": "image/jpeg", ".jpeg": "image/jpeg", ".gif": "image/gif", ".webp": "image/webp"}
    if ext not in media_types:
        return f"Error: Unsupported format: {ext}. Supported: png, jpg, gif, webp"
    
    size = os.path.getsize(path)
    if size > 20 * 1024 * 1024:
        return f"Error: Image too large: {size / (1024*1024):.1f}MB (max 20MB)"
    
    return f"Image loaded: {path} ({size:,} bytes, {ext})"

# ============== TODOS ==============

def tool_todo_write(args: Dict) -> str:
    """Update todo list."""
    global _TODOS
    _TODOS = args["todos"]
    lines = ["Todo List Updated:"]
    for t in _TODOS:
        icon = {"pending": "[ ]", "in_progress": "[>]", "completed": "[x]"}.get(t.get("status"), "[?]")
        lines.append(f"  {icon} {t.get('content', 'Unknown')}")
    return "\n".join(lines)

def tool_todo_read(args: Dict) -> str:
    """Read current todo list."""
    if not _TODOS:
        return "No todos."
    lines = ["Current Todos:"]
    for t in _TODOS:
        icon = {"pending": "[ ]", "in_progress": "[>]", "completed": "[x]"}.get(t.get("status"), "[?]")
        lines.append(f"  {icon} {t.get('content', 'Unknown')}")
    return "\n".join(lines)

# ============== TOOL REGISTRY ==============

TOOLS = {
    "read_file": (tool_read_file, False, "Read file contents with line numbers",
        {"type": "object", "properties": {"file_path": {"type": "string", "description": "Path to file"}, "offset": {"type": "integer", "description": "Start line (0-indexed)"}, "limit": {"type": "integer", "description": "Max lines (default 500)"}}, "required": ["file_path"]}),
    
    "write_file": (tool_write_file, True, "Write content to file. Must read first if exists.",
        {"type": "object", "properties": {"file_path": {"type": "string"}, "content": {"type": "string"}}, "required": ["file_path", "content"]}),
    
    "edit_file": (tool_edit_file, True, "Edit file by replacing EXACT string match. Must read first.",
        {"type": "object", "properties": {"file_path": {"type": "string"}, "old_string": {"type": "string", "description": "Exact text to replace"}, "new_string": {"type": "string"}, "replace_all": {"type": "boolean", "description": "Replace all occurrences"}}, "required": ["file_path", "old_string", "new_string"]}),
    
    "glob": (tool_glob, False, "Find files by glob pattern (e.g., '**/*.py')",
        {"type": "object", "properties": {"pattern": {"type": "string"}, "path": {"type": "string", "description": "Directory to search"}}, "required": ["pattern"]}),
    
    "grep": (tool_grep, False, "Search file contents with regex",
        {"type": "object", "properties": {"pattern": {"type": "string", "description": "Regex pattern"}, "path": {"type": "string"}, "glob": {"type": "string", "description": "Filter files"}, "case_insensitive": {"type": "boolean"}}, "required": ["pattern"]}),
    
    "list_dir": (tool_list_dir, False, "List directory contents",
        {"type": "object", "properties": {"path": {"type": "string"}}, "required": []}),
    
    "bash": (tool_bash, True, "Run shell command. Use for git, pip, scripts.",
        {"type": "object", "properties": {"command": {"type": "string"}, "timeout": {"type": "integer", "description": "Timeout seconds (max 600)"}}, "required": ["command"]}),
    
    "python_exec": (tool_python_exec, True, "Execute Python code. Use for data processing, calculations, file generation.",
        {"type": "object", "properties": {"code": {"type": "string", "description": "Python code"}, "timeout": {"type": "integer", "description": "Timeout seconds (max 300)"}}, "required": ["code"]}),
    
    "create_word": (tool_create_word, True, "Create Word document (.docx)",
        {"type": "object", "properties": {"filepath": {"type": "string"}, "content": {"type": "string", "description": "Paragraphs separated by \\n\\n"}, "title": {"type": "string"}}, "required": ["filepath", "content"]}),
    
    "create_excel": (tool_create_excel, True, "Create Excel spreadsheet (.xlsx)",
        {"type": "object", "properties": {"filepath": {"type": "string"}, "data": {"type": "array", "description": "List of dicts [{\"col\": \"val\"}]"}, "sheet_name": {"type": "string"}}, "required": ["filepath", "data"]}),
    
    "create_markdown": (tool_create_markdown, True, "Create Markdown file (.md)",
        {"type": "object", "properties": {"filepath": {"type": "string"}, "content": {"type": "string"}}, "required": ["filepath", "content"]}),
    
    "view_image": (tool_view_image, False, "View image (PNG, JPG, GIF, WebP)",
        {"type": "object", "properties": {"file_path": {"type": "string"}}, "required": ["file_path"]}),
    
    "todo_write": (tool_todo_write, False, "Update task list for tracking multi-step work",
        {"type": "object", "properties": {"todos": {"type": "array", "items": {"type": "object", "properties": {"content": {"type": "string"}, "status": {"type": "string", "enum": ["pending", "in_progress", "completed"]}, "activeForm": {"type": "string"}}}}}, "required": ["todos"]}),
    
    "todo_read": (tool_todo_read, False, "Read current task list",
        {"type": "object", "properties": {}, "required": []}),
}

def get_tool_definitions() -> List[Dict]:
    """Get tool definitions for Bedrock API."""
    return [{"name": k, "description": v[2], "input_schema": v[3]} for k, v in TOOLS.items()]

print(f"Loaded {len(TOOLS)} tools:")
for name, (_, needs_approval, desc, _) in TOOLS.items():
    approval = "(approval)" if needs_approval else ""
    print(f"  - {name} {approval}")

In [ ]:
# ============================================================
# CELL 9: SYSTEM PROMPT
# ============================================================

SYSTEM_PROMPT = """You are SageMaker Coding Agent, an AI assistant for software engineering in AWS SageMaker.

# Style
- Be concise. Use markdown. No emojis unless asked.
- Prefer editing existing files over creating new ones.
- Be technically accurate. Disagree when necessary.

# Task Management
Use todo_write frequently to track tasks. Mark todos completed immediately when done. Only ONE task should be in_progress at a time.

# Doing Tasks
- ALWAYS read a file before editing (edit_file fails otherwise)
- old_string in edit_file must be EXACT match
- Use specialized tools over bash: read_file (not cat), edit_file (not sed), glob (not find), grep (not grep)
- Reserve bash for: git, pip/npm, running scripts

# Document Creation
You can create Word (.docx), Excel (.xlsx), and Markdown (.md) files:
- create_word: Use for formal documents, reports
- create_excel: Use for tabular data, spreadsheets. Data format: list of dicts
- create_markdown: Use for documentation

# Python Execution
Use python_exec for:
- Data processing and analysis
- Complex calculations
- Custom file generation
- Any scripting task

# Code References
Use pattern `file_path:line_number` when referencing code.

# Security
- Workspace boundary enforced (cannot access outside project)
- Dangerous commands blocked (rm -rf, sudo, curl|bash)
- Write operations need user approval
- All actions logged to audit trail
"""

print(f"System prompt ready ({len(SYSTEM_PROMPT)} chars, ~{len(SYSTEM_PROMPT)//4} tokens)")

In [ ]:
# ============================================================
# CELL 10: AGENT LOOP
# ============================================================

class Agent:
    """Main agent loop with ReAct pattern, history trimming, and doom loop detection."""
    
    def __init__(self, client: BedrockClient, session_id: str = None, on_approval: Callable = None):
        self.client = client
        self.session_id = session_id or datetime.now().strftime("%Y%m%d_%H%M%S")
        self.messages = []
        self.on_approval = on_approval
        self.tool_history = deque(maxlen=10)
    
    def run(self, user_message: str, output_fn: Callable = print) -> str:
        """Run agent loop until completion or max turns."""
        self.messages.append({"role": "user", "content": user_message})
        AUDIT.log(self.session_id, "user_message", parameters={"message": user_message[:200]})
        
        for turn in range(CONFIG.max_turns):
            # Check context usage
            warning = CONTEXT.check_and_warn(self.messages)
            if warning:
                output_fn(warning)
            
            # Trim history if too long
            if len(self.messages) > CONFIG.max_history:
                self.messages = [{"role": "user", "content": "[Earlier messages trimmed]"}] + self.messages[-CONFIG.max_history:]
            
            # Call LLM
            try:
                response = self.client.chat(self.messages, SYSTEM_PROMPT, get_tool_definitions(), CONFIG.max_tokens)
            except Exception as e:
                error_msg = f"Error calling Bedrock: {e}"
                output_fn(error_msg)
                AUDIT.log(self.session_id, "error", result_summary=str(e))
                return error_msg
            
            # Output text
            if response.text:
                output_fn(response.text)
            
            # No tool calls = done
            if not response.tool_calls:
                AUDIT.log(self.session_id, "response", result_summary=response.text[:200] if response.text else "")
                return response.text or ""
            
            # Check doom loop
            for tc in response.tool_calls:
                key = (tc.name, str(tc.input))
                if sum(1 for h in self.tool_history if h == key) >= 3:
                    output_fn("[Warning: Repetitive tool calls detected, stopping]")
                    return response.text or ""
                self.tool_history.append(key)
            
            # Build assistant message
            assistant_content = []
            if response.text:
                assistant_content.append({"type": "text", "text": response.text})
            for tc in response.tool_calls:
                assistant_content.append({"type": "tool_use", "id": tc.id, "name": tc.name, "input": tc.input})
            self.messages.append({"role": "assistant", "content": assistant_content})
            
            # Execute tools
            tool_results = []
            for tc in response.tool_calls:
                tool_info = TOOLS.get(tc.name)
                if not tool_info:
                    tool_results.append({"type": "tool_result", "tool_use_id": tc.id, "content": f"Unknown tool: {tc.name}"})
                    continue
                
                func, needs_approval, _, _ = tool_info
                
                # Check approval
                if needs_approval and self.on_approval:
                    approved = self.on_approval(tc.name, tc.input)
                    AUDIT.log(self.session_id, "approval_request", tc.name, tc.input, 
                             "Approved" if approved else "Denied", approved)
                    if not approved:
                        tool_results.append({"type": "tool_result", "tool_use_id": tc.id, "content": "User denied permission"})
                        continue
                
                # Execute tool
                output_fn(f"[Calling {tc.name}...]")
                try:
                    result = func(tc.input)
                except Exception as e:
                    result = f"Error: {e}"
                
                # Truncate result
                result = SECURITY.truncate_output(result)
                AUDIT.log(self.session_id, "tool_call", tc.name, tc.input, result[:200])
                tool_results.append({"type": "tool_result", "tool_use_id": tc.id, "content": result})
            
            self.messages.append({"role": "user", "content": tool_results})
        
        output_fn(f"[Reached max turns ({CONFIG.max_turns})]")
        return response.text if response else ""
    
    def reset(self):
        """Reset conversation."""
        global _TODOS, _FILES_READ
        self.messages = []
        self.tool_history.clear()
        _TODOS = []
        _FILES_READ = set()
        CONTEXT.reset()

print("Agent class ready")

In [ ]:
# ============================================================
# CELL 11: CHAT INTERFACE
# ============================================================

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# UI Components
chat_output = widgets.Output(layout=widgets.Layout(height='400px', overflow_y='auto', border='1px solid #ccc'))
input_box = widgets.Textarea(placeholder='Type your message...', layout=widgets.Layout(width='100%', height='80px'))
send_btn = widgets.Button(description='Send', button_style='primary', icon='paper-plane')
clear_btn = widgets.Button(description='Clear', button_style='warning', icon='trash')
save_btn = widgets.Button(description='Save Session', button_style='info', icon='save')
status_html = widgets.HTML(value='<span style="color:green"><b>Ready</b></span>')

# Approval dialog
approval_output = widgets.Output()
approve_btn = widgets.Button(description='Approve', button_style='success', icon='check')
deny_btn = widgets.Button(description='Deny', button_style='danger', icon='times')
approval_box = widgets.VBox([approval_output, widgets.HBox([approve_btn, deny_btn])])
approval_box.layout.display = 'none'

# Session selector
session_dropdown = widgets.Dropdown(description='Session:', options=[('New Session', None)], layout=widgets.Layout(width='300px'))
load_btn = widgets.Button(description='Load', button_style='', icon='folder-open')

pending_approval = {"result": None}
current_session = {"session": None, "agent": None}

def update_session_list():
    sessions = SESSIONS.list_sessions()
    options = [('New Session', None)] + [(f"{s['title'][:30]} ({s['id']})", s['id']) for s in sessions[:10]]
    session_dropdown.options = options

def add_message(role: str, content: str):
    with chat_output:
        ts = datetime.now().strftime('%H:%M:%S')
        content_escaped = str(content).replace('<', '&lt;').replace('>', '&gt;')
        if role == 'user':
            display(HTML(f'<div style="background:#e3f2fd;padding:10px;margin:5px;border-radius:5px;"><b>[{ts}] You:</b><br>{content_escaped}</div>'))
        elif role == 'assistant':
            display(HTML(f'<div style="background:#f5f5f5;padding:10px;margin:5px;border-radius:5px;"><b>[{ts}] Agent:</b><br><pre style="white-space:pre-wrap;font-family:monospace;">{content_escaped}</pre></div>'))
        elif role == 'tool':
            display(HTML(f'<div style="background:#fff3e0;padding:8px;margin:3px;border-radius:5px;font-size:12px;"><b>{content_escaped}</b></div>'))
        elif role == 'system':
            display(HTML(f'<div style="background:#ffebee;padding:8px;margin:3px;border-radius:5px;font-size:12px;color:#c62828;"><b>{content_escaped}</b></div>'))

def request_approval(tool_name: str, tool_input: Dict) -> bool:
    pending_approval["result"] = None
    with approval_output:
        clear_output()
        input_str = json.dumps(tool_input, indent=2, default=str)[:500]
        display(HTML(f'<div style="padding:10px;background:#fff8e1;border-radius:5px;"><h4>Approval Required</h4><p><b>Tool:</b> {tool_name}</p><pre style="font-size:11px;">{input_str}</pre></div>'))
    approval_box.layout.display = 'block'
    send_btn.disabled = True
    
    while pending_approval["result"] is None:
        time.sleep(0.1)
    
    approval_box.layout.display = 'none'
    send_btn.disabled = False
    return pending_approval["result"]

def on_approve(b):
    pending_approval["result"] = True
    add_message('tool', 'Approved')

def on_deny(b):
    pending_approval["result"] = False
    add_message('tool', 'Denied')

approve_btn.on_click(on_approve)
deny_btn.on_click(on_deny)

def on_send(b):
    msg = input_box.value.strip()
    if not msg:
        return
    input_box.value = ''
    
    if current_session["agent"] is None:
        session = SESSIONS.create(title=msg[:50])
        current_session["session"] = session
        current_session["agent"] = Agent(CLIENT, session.id, on_approval=request_approval)
        update_session_list()
    
    add_message('user', msg)
    status_html.value = '<span style="color:orange"><b>Processing...</b></span>'
    
    def output_fn(text):
        if text.startswith('[Calling') or text.startswith('[Warning') or text.startswith('[!') or text.startswith('[i]'):
            add_message('tool', text)
        else:
            add_message('assistant', text)
    
    try:
        current_session["agent"].run(msg, output_fn)
    except Exception as e:
        add_message('system', f'Error: {e}')
    
    if current_session["agent"]:
        usage = CONTEXT.get_usage(current_session["agent"].messages)
        color = {'normal': 'green', 'medium': 'orange', 'high': 'red', 'critical': 'darkred'}[usage['level']]
        status_html.value = f'<span style="color:{color}"><b>Ready ({usage["percent"]*100:.0f}% context)</b></span>'

def on_clear(b):
    global _TODOS, _FILES_READ
    if current_session["agent"]:
        current_session["agent"].reset()
    current_session["session"] = None
    current_session["agent"] = None
    _TODOS = []
    _FILES_READ = set()
    chat_output.clear_output()
    with chat_output:
        display(HTML('<p><i>Chat cleared. Start a new conversation.</i></p>'))
    status_html.value = '<span style="color:green"><b>Ready</b></span>'

def on_save(b):
    if current_session["session"] and current_session["agent"]:
        current_session["session"].messages = current_session["agent"].messages
        SESSIONS.save(current_session["session"])
        add_message('system', f'Session saved: {current_session["session"].id}')
        update_session_list()
    else:
        add_message('system', 'No session to save')

def on_load(b):
    session_id = session_dropdown.value
    if session_id:
        session = SESSIONS.load(session_id)
        if session:
            current_session["session"] = session
            current_session["agent"] = Agent(CLIENT, session.id, on_approval=request_approval)
            current_session["agent"].messages = session.messages
            chat_output.clear_output()
            add_message('system', f'Loaded session: {session.title}')

send_btn.on_click(on_send)
clear_btn.on_click(on_clear)
save_btn.on_click(on_save)
load_btn.on_click(on_load)

# Layout
header = widgets.HTML('<h2>SageMaker Coding Agent</h2><p style="color:gray;">Secure AI assistant for software engineering | 14 tools | Session management</p>')
session_row = widgets.HBox([session_dropdown, load_btn])
buttons = widgets.HBox([send_btn, clear_btn, save_btn, status_html])
ui = widgets.VBox([header, session_row, chat_output, approval_box, input_box, buttons])

update_session_list()
with chat_output:
    display(HTML('<p><i>Welcome! Type a message and click Send.</i></p>'))

display(ui)

---

## Usage Examples

### File Operations
```
"List all Python files"
"Read config.py"
"Create a hello.py file with a hello world function"
"Edit config.py and change the region to us-east-1"
```

### Search
```
"Search for 'TODO' in all Python files"
"Find files containing 'def main'"
```

### Shell Commands
```
"Run git status"
"Run pip list"
```

### Documents
```
"Create a Word document summarizing this project"
"Create an Excel file with columns: name, age, city and add 3 rows"
"Create a README.md for this folder"
```

### Python Execution
```
"Run Python code to calculate fibonacci(10)"
"Generate a CSV file with random data using pandas"
```

---

## Feature Comparison: Single-File vs Multi-File

| Feature | Single-File | Multi-File |
|---------|-------------|------------|
| Tools | 14 | 14 |
| Security | Full | Full |
| Audit Logging | Yes | Yes |
| Session Management | Yes | Yes |
| Context Warnings | Yes | Yes |
| Approval System | Yes | Yes |
| Mock Mode | Yes | Yes |

---

## Configuration

Edit the `Config` class in Cell 2:

| Setting | Default | Description |
|---------|---------|-------------|
| `region` | ap-southeast-2 | AWS region (Sydney) |
| `model_id` | claude-3-5-sonnet | Bedrock model |
| `mock_mode` | False | Test without API |
| `max_turns` | 30 | Max tool loop iterations |
| `max_history` | 20 | Messages before trimming |

---

## Troubleshooting

| Issue | Solution |
|-------|----------|
| "Access denied" | Enable Claude in Bedrock console |
| "Throttling" | Hit token quota - wait or request increase |
| "Path outside workspace" | Only files in current folder accessible |
| "Must read file first" | Use read_file before edit_file |
| "Command blocked" | Security blocks dangerous commands |
| Test without API | Set `mock_mode = True` in Config |